In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

if os.environ['GOOGLE_API_KEY']:
    print("API Key found.")
else:
    raise EnvironmentError("API Key not found in environment variables.")

API Key found.


In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(
    model="gemini-3-flash-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2
)

In [3]:
messages = [
    (
        "system",
        "You are a helpful assistant that helps me in learning Generative AI models and LLMs and RAGs",
    ),
    ("What is RAG in LLMs?"),
]
ai_msg = model.invoke(messages)
ai_msg

AIMessage(content=[{'type': 'text', 'text': '**RAG** stands for **Retrieval-Augmented Generation**. It is currently one of the most popular and practical architectures used in the world of Generative AI.\n\nTo understand it simply: **RAG gives an LLM an "Open Book" to look at before it answers a question.**\n\n---\n\n### 1. The Problem RAG Solves\nStandard LLMs (like GPT-4 or Claude) have two major limitations:\n1.  **Knowledge Cutoff:** They only know what they were trained on up to a certain date. They don\'t know about today\'s news or your private files.\n2.  **Hallucinations:** When an LLM doesn\'t know an answer, it often "hallucinates" (makes up a confident-sounding but false answer).\n\n### 2. How RAG Works (The 3 Steps)\nInstead of relying solely on the LLM\'s internal memory, RAG follows a three-step process:\n\n#### **Step 1: Retrieval**\nWhen you ask a question, the system searches through a specific dataset (like your company’s PDFs, a product manual, or a live news feed) 

In [4]:
print(ai_msg.content[0]['text'])

**RAG** stands for **Retrieval-Augmented Generation**. It is currently one of the most popular and practical architectures used in the world of Generative AI.

To understand it simply: **RAG gives an LLM an "Open Book" to look at before it answers a question.**

---

### 1. The Problem RAG Solves
Standard LLMs (like GPT-4 or Claude) have two major limitations:
1.  **Knowledge Cutoff:** They only know what they were trained on up to a certain date. They don't know about today's news or your private files.
2.  **Hallucinations:** When an LLM doesn't know an answer, it often "hallucinates" (makes up a confident-sounding but false answer).

### 2. How RAG Works (The 3 Steps)
Instead of relying solely on the LLM's internal memory, RAG follows a three-step process:

#### **Step 1: Retrieval**
When you ask a question, the system searches through a specific dataset (like your company’s PDFs, a product manual, or a live news feed) to find the most relevant information.

#### **Step 2: Augmentat

# RAG Implementation with our own data

## Step 1 - Preparing documents for text

In [5]:
from langchain_core.documents import Document

my_info = '''
My name is Vishu Yadav.
i am a data engineer.
i am looking to learn more about gen ai models so that i can build rags and use it in my projects.'''

docs = [Document(page_content=my_info, metadata={'source': 'my_info.txt', 'documentId': 'doc1'})]
docs

[Document(metadata={'source': 'my_info.txt', 'documentId': 'doc1'}, page_content='\nMy name is Vishu Yadav.\ni am a data engineer.\ni am looking to learn more about gen ai models so that i can build rags and use it in my projects.')]

## Step 2 - Splitting the documents into chunks

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=15
)

chunks=splitter.split_documents(docs)

chunks

[Document(metadata={'source': 'my_info.txt', 'documentId': 'doc1'}, page_content='My name is Vishu Yadav.\ni am a data engineer.'),
 Document(metadata={'source': 'my_info.txt', 'documentId': 'doc1'}, page_content='i am looking to learn more about gen ai models so'),
 Document(metadata={'source': 'my_info.txt', 'documentId': 'doc1'}, page_content='ai models so that i can build rags and use it in'),
 Document(metadata={'source': 'my_info.txt', 'documentId': 'doc1'}, page_content='and use it in my projects.')]

## Step 3 - Create embedding for these chunks

In [7]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

vector = embeddings.embed_query("what is my name?")
vector

[-0.029319886,
 -0.0026982331,
 0.0067081978,
 -0.060882173,
 -0.013955684,
 0.017710684,
 -0.009455445,
 -0.023264581,
 -0.003336156,
 0.011691645,
 0.0019990937,
 0.00042126697,
 -0.01597051,
 0.025979493,
 0.13211948,
 -0.0046519684,
 -0.0058029024,
 -0.004228794,
 -0.0048241275,
 -0.012066664,
 -0.006569005,
 0.013659688,
 0.0103525845,
 -0.02041811,
 0.00975222,
 0.01191967,
 0.011656213,
 -0.022601118,
 0.03563319,
 0.00097284955,
 -0.019769773,
 -0.005865943,
 -0.026990661,
 0.015106303,
 0.0068694153,
 0.0053420295,
 -0.019248437,
 -0.02610541,
 -0.019513993,
 0.004119565,
 0.0031329286,
 -0.013866344,
 0.0066066957,
 0.00763955,
 0.0068795625,
 -0.0043472867,
 -0.01117298,
 -0.02395778,
 -0.0020839588,
 0.029832887,
 0.0044775587,
 0.026541,
 -0.025782127,
 -0.14203648,
 -0.00072361715,
 -0.0064168037,
 0.016414354,
 -0.01043748,
 0.0052591967,
 -0.0064822305,
 -0.013602413,
 -0.002091699,
 -0.042178933,
 -0.025658248,
 0.0057811253,
 0.008894629,
 0.00635629,
 0.03807071,
 0.

## Step 4 - Create and store embeddings in vector database/store

In [8]:
from langchain_community.vectorstores import Chroma

vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

## Step 5 - Semantic search - Querying the vector store

In [13]:
output1 = vector_store.similarity_search("What is my name?", k=2)
output1

[Document(metadata={'documentId': 'doc1', 'source': 'my_info.txt'}, page_content='My name is Vishu Yadav.\ni am a data engineer.'),
 Document(metadata={'documentId': 'doc1', 'source': 'my_info.txt'}, page_content='and use it in my projects.')]

In [14]:
output2 = vector_store.similarity_search("What do i do?", k=2)
output2

[Document(metadata={'source': 'my_info.txt', 'documentId': 'doc1'}, page_content='My name is Vishu Yadav.\ni am a data engineer.'),
 Document(metadata={'documentId': 'doc1', 'source': 'my_info.txt'}, page_content='and use it in my projects.')]

In [15]:
output3 = vector_store.similarity_search("What i want to do here?", k=2)
output3

[Document(metadata={'documentId': 'doc1', 'source': 'my_info.txt'}, page_content='and use it in my projects.'),
 Document(metadata={'documentId': 'doc1', 'source': 'my_info.txt'}, page_content='i am looking to learn more about gen ai models so')]

In [20]:
ai_msg = model.invoke(f"What is my name? you can use the below context to answer the question: {output1}")
print(ai_msg.content[0]['text'])

Based on the context provided, your name is **Vishu Yadav**.


In [ ]:
ai_msg = model.invoke(f"What work do i do? you can use the below context to answer the question: {output2}")
print(ai_msg.content[0]['text'])

Based on the context provided, you are a **data engineer**.


In [19]:
ai_msg = model.invoke(f"What am i doing here? what is my purpose? you can use the below context to answer the question: {output3}")
print(ai_msg.content[0]['text'])

Based on the context provided, you are here to **learn more about Generative AI models**. Your purpose is to gain this knowledge so that you can **use it in your projects**.


In [21]:
ai_msg = model.invoke(f"What is my name? you can use the below context to answer the question: {output3}")
print(ai_msg.content[0]['text'])

Based on the context provided, your name is not mentioned. The documents only state that you are looking to learn more about Gen AI models and use them in your projects.
